# Erwartungswert

In [120]:
import qib
from qib.operator import FieldOperator, FieldOperatorTerm, IFOType, IFODesc
import fermitensor as ftn # after installing fermitensor (installed with kernel 3.10.11)

import numpy as np
from scipy import sparse
from typing import Sequence

### Hamiltonian for TTN

In [109]:
L = 8
L_1 = L//2
A = [0, L_1]
B = [L_1, L]

latt = qib.lattice.FullyConnectedLattice((L,))
field = qib.field.Field(qib.field.ParticleType.FERMION, latt)

# create MolecularHamiltonian Object
# H = construct_random_molecular_hamiltonian(L)

#### Create all components in (11)-(13)

In [114]:
# TODO dimensionen korrigieren mit rüberkopieren
def get_P_as_field_operator(H, L, i, j):
    V = FieldOperatorTerm([IFODesc(H.field, IFOType.FERMI_ANNIHIL),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            get_range(H.vint[i, j, :, :], L).transpose((1, 0)))
    return FieldOperator([V])
    
def get_Q_as_field_operator(H, L, i, j):
    V = FieldOperatorTerm([IFODesc(H.field, IFOType.FERMI_CREATE),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            get_range(H.vint[i, :, j, :] - H.vint[i, :, :, j], L))
    return FieldOperator([V])

def get_S_as_field_operator(H, L, i):
    # kinetic hopping term
    T = FieldOperatorTerm([IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            get_range(H.tkin[i, :], L))
    # interaction term
    V = FieldOperatorTerm([IFODesc(H.field, IFOType.FERMI_CREATE),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            get_range(H.vint[i, :, :, :].transpose((0, 2, 1)), L))
    return FieldOperator([T, V])

In [ ]:
# TODO
def get_field_operator(types: Sequence[IFOType]):
    ret = []
    for type in types:
        ret.append(IFODesc(H.field, type))
    return FieldOperator([FieldOperatorTerm(ret, None)])

In [115]:
A = np.random.rand(10,10,10)
print(A[1,:,])

[[0.60369427 0.01451818 0.48871096 0.68104935 0.40297469 0.19731234
  0.01946547 0.08985161 0.33226929 0.23085111]
 [0.80643004 0.98246865 0.11307045 0.71115513 0.35831733 0.80669686
  0.50786356 0.17674889 0.1946069  0.67640056]
 [0.99289726 0.13103566 0.42109869 0.82963589 0.66691456 0.74727473
  0.838949   0.83946087 0.90217137 0.20885248]
 [0.65136852 0.12346954 0.82907637 0.33128505 0.4310094  0.47684838
  0.11796042 0.48438279 0.14350889 0.00877944]
 [0.56759574 0.51600064 0.36410223 0.95689651 0.78013571 0.39119733
  0.89353832 0.50424166 0.05865608 0.33231306]
 [0.85775969 0.07459333 0.51050735 0.04715188 0.60730775 0.7738765
  0.10678361 0.54321632 0.0017879  0.01296924]
 [0.55917489 0.98619557 0.14458022 0.16043894 0.31980843 0.63194515
  0.87308105 0.09139047 0.09047562 0.60097576]
 [0.20781769 0.65055186 0.08451153 0.37424413 0.68569777 0.18726081
  0.50323331 0.82460119 0.17191083 0.03914721]
 [0.14750509 0.58374151 0.33123001 0.35152883 0.81555028 0.51093901
  0.31331033 

In [116]:
def multiply(P: FieldOperator, Q: FieldOperator):
    # TODO multiplying two Field Operators
    ret = []

    for p_term in P.terms:
        for q_term in Q.terms:
            opdesc = None
            coeffs = p_term.coeffs * q_term.coeffs # TODO element wise right?
            ret.append(FieldOperatorTerm(opdesc, coeffs))

    return FieldOperator(ret)

In [118]:
def get_H_AB_as_field_operator(H, A, B):
    # acc. to equ. (10) in paper
    S_i = get_field_operator([IFOType.FERMI_CREATE]).as_matrix() @ get_S_as_field_operator(H, B, i).as_matrix() # TODO Matrixmultiplikation zuerst
    S_j = get_S_as_field_operator(H, A, IFOType.FERMI_CREATE, B)
    Q_ii = get_Q_as_field_operator(H, B, IFOType.FERMI_CREATE, A, IFOType.FERMI_ANNIHIL, A) # TODO same index, maybe go back to with i and j
    
    # TODO i > j in A
    P_ij = get_P_as_field_operator(H, B, IFOType.FERMI_ANNIHIL, A, IFOType.FERMI_ANNIHIL, A) # TODO adjoint in the middle
    P_ij_adj = adjoint(P_ij)
    Q_ij = get_Q_as_field_operator(H, B, IFOType.FERMI_CREATE, A, IFOType.FERMI_ANNIHIL, A)
    Q_ij_adj = adjoint(Q_ij)

    return FieldOperator([S_i, S_j, Q_ii, P_ij, P_ij_adj, Q_ij, Q_ij_adj])

In [119]:
H_AB = get_H_AB_as_field_operator(H, A, B)

TypeError: get_S_as_field_operator() takes 3 positional arguments but 4 were given

In [ ]:
# TODO final Hamiltonian according to equ. (9)
H = H_A + H_B + H_AB

In [94]:
# compare new hamiltonian with other (einfach abziehen voneinander) norm der differenz, sparse linalg norm, sollte 0 sein


### Abandoned Stuff

In [ ]:
def get_P_as_field_operator(H, L, a: IFOType, A, b: IFOType, B):
    V = FieldOperatorTerm([IFODesc(H.field, a),
                            IFODesc(H.field, b),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            H.vint[A[0]:A[1], B[0]:B[1], L[0]:L[1], L[0]:L[1]].transpose((0, 1, 3, 2)))
    return FieldOperator([V])
    
def get_Q_as_field_operator(H, L, a: IFOType, A, b: IFOType, B):
    V = FieldOperatorTerm([IFODesc(H.field, a),
                            IFODesc(H.field, b),
                            IFODesc(H.field, IFOType.FERMI_CREATE),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            H.vint[A[0]:A[1], L[0]:L[1], B[0]:B[1], L[0]:L[1]].transpose((0, 2, 1, 3))
                              - H.vint[A[0]:A[1], L[0]:L[1], L[0]:L[1], B[0]:B[1]].transpose((0, 2, 3, 1)))
    return FieldOperator([V])

def get_S_as_field_operator(H, L, a: IFOType, A):
    # kinetic hopping term
    T = FieldOperatorTerm([IFODesc(H.field, a),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            H.tkin[A[0]:A[1], L[0]:L[1]])
    # interaction term
    V = FieldOperatorTerm([IFODesc(H.field, a),
                            IFODesc(H.field, IFOType.FERMI_CREATE),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL),
                            IFODesc(H.field, IFOType.FERMI_ANNIHIL)],
                            H.vint[A[0]:A[1], L[0]:L[1], L[0]:L[1], L[0]:L[1]].transpose((0, 1, 3, 2)))
    return FieldOperator([T, V])

In [79]:
def prepend_IFO(P: FieldOperator, a: IFOType, L):
    # prepend IFO to a Field Operator
    # TODO how does that work with the dimensions of coeff???
    for term in P.terms:
        term.opdesc = (IFODesc(term.opdesc[0].field, a),) + term.opdesc # TODO Case possible with empty list?

In [99]:
# a_i^At S_i^B i in A
S = get_S_as_field_operator(H, B, IFOType.FERMI_CREATE, A)
for term in S.terms:
    print(term)
    print(term.opdesc)
print(S.is_hermitian())

(<qib.operator.field_operator.IFODesc object at 0x00000202FD873400>, <qib.operator.field_operator.IFODesc object at 0x00000202FD8738B0>)
(<qib.operator.field_operator.IFODesc object at 0x00000202FD872AA0>, <qib.operator.field_operator.IFODesc object at 0x00000202FD872800>, <qib.operator.field_operator.IFODesc object at 0x00000202FD872F50>, <qib.operator.field_operator.IFODesc object at 0x00000202FD8734F0>)


NotImplementedError: 